<a href="https://colab.research.google.com/github/AndreyKuratov/ChipTip/blob/main/Data_upload_LISA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
import torch
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import yaml
import pandas as pd
from shutil import copyfile
from PIL import Image
import pathlib
import glob

# **1. Скачивание и разархиироание данных**

Поключить свой диск с разшаренной папкой ФФ

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Скачивание базы из https://www.kaggle.com/

In [4]:
 ! pip install -q kaggle

In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alexwatcher","key":"95bba53e56226f2227527b42560bed76"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
! kaggle datasets download -d mbornoe/lisa-traffic-light-dataset

Копирование для хранения на диск

In [ ]:
!cp /content/lisa-traffic-light-dataset.zip /content/drive/MyDrive/TL_project

Распаковка в директорию для работы


In [ ]:
!unzip /content/lisa-traffic-light-dataset.zip -d /content/project_data/

In [51]:
!rm -rf /content/project_dadta/

In [52]:
!rm -rf /content/project_data/nightSequence1
!rm -rf /content/project_data/nightSequence2
!rm -rf /content/project_data/nightTrain
!rm -rf /content/project_data/sample-dayClip6
!rm -rf /content/project_data/sample-nightClip1

In [53]:
!rm -rf /content/project_data/Annotations/Annotations/nightSequence1
!rm -rf /content/project_data/Annotations/Annotations/nightSequence2
!rm -rf /content/project_data/Annotations/Annotations/nightTrain
!rm -rf /content/project_data/Annotations/Annotations/sample-dayClip6
!rm -rf /content/project_data/Annotations/Annotations/sample-nightClip1

Команды консоли которые могу понодобиться

In [ ]:
#!unzip /content/Lara_UrbanSeq1_JPG.zip
#!rm -rf /content/drive/MyDrive/TL_project/rgb
#!cp -r /content/drive/MyDrive/TL_project/Lara3D_UrbanSeq1_JPG /content/Lara_data/

# **2. Работа с метками**


### Просмотр меток.
 Сравнить на выводе чем отлиыаются BOX and BULB

In [24]:
path_to_labels01 = '/content/project_dadta/Annotations/Annotations/daySequence1/frameAnnotationsBOX.csv'
path_to_labels02 = '/content/project_dadta/Annotations/Annotations/daySequence1/frameAnnotationsBULB.csv'
data01 = pd.read_csv(path_to_labels01,sep=';')
data02 = pd.read_csv(path_to_labels02,sep=';')

In [77]:
data01.head(2)
data01['Annotation tag'].unique()

array(['stop', 'go', 'warning', 'warningLeft', 'stopLeft'], dtype=object)

In [26]:
data02.head(2)

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin file,Origin frame number,Origin track,Origin track frame number
0,dayTest/daySequence1--00000.jpg,stop,710,481,714,486,dayTest/daySequence1/Day1EveningShutter0.00020...,0,dayTest/daySequence1/Day1EveningShutter0.00020...,0
1,dayTest/daySequence1--00001.jpg,stop,710,478,715,483,dayTest/daySequence1/Day1EveningShutter0.00020...,1,dayTest/daySequence1/Day1EveningShutter0.00020...,1


## Создает список со всеми файлмаи



In [143]:
def list_of_files(path_walk,ftype='txt'):
    list_path_files = []
    for root, dirs, files in os.walk(path_walk):
      for filename in files:
        if (filename[-len(ftype):] == ftype):
          tot_path = root+'/' + filename 
          list_path_files.append(tot_path)
    return list_path_files

def copyJpgfromList(dirPath,list_of_files):
    path = pathlib.Path(dirPath)
    path.parent.mkdir(parents=True, exist_ok=True)
    if not( os.path.isdir(dirPath)):
      os.mkdir(dirPath)
    for ipathFile in  list_of_files:
      copyfile(ipathFile, dirPath+ipathFile.split('/')[-1])

def crtTxtFiles(path_save,dataframe, ordLabels=['fileName01','lbl','cntX','cntY','Width','Height']):
    path = pathlib.Path(path_save)
    path.parent.mkdir(parents=True, exist_ok=True) 
    if not( os.path.isdir(path_save)):
      os.mkdir(path_save)

    unique_names = dataframe[ordLabels[0]].unique()
    for iname in unique_names:
      savePath = path_save+iname[:-3]+'txt'
      df = dataframe[dataframe['fileName01']==iname]
      with open(savePath, 'w') as f:
        for index, row in df.iterrows():
          line = str(row[ordLabels[1]])+' '+str(row[ordLabels[2]])+' '+str(row[ordLabels[3]])+' '+str(row[ordLabels[4]])+' '+str(row[ordLabels[5]])
          print(line, file=f)
        f.close()

def compareTextJpg(path_save = '/content/'):
  path_walk = path_save
  count_txt = 0
  count_jpg = 0 
  for root, dirs, files in os.walk(path_walk):
      for filename in files:
        if (filename[-3:] == 'jpg'):
          count_jpg+=1
        if (filename[-3:] == 'txt'):
          count_txt+=1
        path_txt = root+filename[:-3]+'txt'
        path_jpg =  root+filename[:-3]+'jpg'
        if ((os.path.exists(path_jpg)))and (not(os.path.exists(path_txt))):
          print('txt jpg',path_txt)
        if (not(os.path.exists(path_jpg)))and ((os.path.exists(path_txt))):
          print('no jpg',path_jpg)
  print(count_jpg,count_txt)


def crtEmptyTxt(path_save,svEmptFlag=True,emptyName='empty.txt'):
    emptyList = []
    for root, dirs, files in os.walk(path_save):
      for filename in files:
        if (filename[-3:] == 'jpg'):
          path_jpg =root+'/'+filename
          path_txt = root+'/'+filename[:-3]+'txt'
          if not( os.path.isfile(path_txt)):
            emptyList.append(filename[:-3]+'txt')
            f = open(path_txt,'w')
            f.close()

    if (svEmptFlag):
      fl = open(path_save+emptyName,'w')
      for lineEmpty in emptyList:
        print(lineEmpty,file=fl)
      fl.close()


    return emptyList

def makeVidoeYolo(path_files,out_video_path,N_frames=10,fps=5):
  img_array = []

  def make_rect(img, coords, height, width,clrLbl):
      x1 = int(coords[0] - coords[2]/2)
      x2 = int(coords[0] + coords[2]/2)
      y1 = int(coords[1] - coords[3]/2)
      y2 = int(coords[1] + coords[3]/2)
      if clrLbl == 0:
        colrTup =  (0,0,0)
      elif clrLbl == 1 :
        colrTup = (0,0,255)
      elif clrLbl == 2 :
        colrTup = (0,255,255) 
      elif clrLbl == 3 :
        colrTup = (0,255,0)
      else: 
        colrTup = (255,0,0)
      img = cv2.rectangle(img,(x2,y2),(x1,y1),
                        colrTup,3)
      return img

  path_proceed = path_files+'/*.jpg'
  for filename in glob.glob(path_proceed)[:N_frames]:
      print(filename)
      img = cv2.imread(filename)
      f = open(filename[:-3]+'txt')
      height, width, layers = img.shape
      size = (width,height)

      for lineData in f.read().split('\n'):
          lineDD = lineData.split(' ') 
          if len(lineDD)>2:
            cl_lbl = lineDD[0]
            coords = [float(lineDD[1])*width, float(lineDD[2])*height,
                      float(lineDD[3])*width, float(lineDD[4])*height]

            img = make_rect(img, coords,height,width,int(cl_lbl))
      f.close()
      img_array.append(img)
  out = cv2.VideoWriter(out_video_path,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
  for i in range(len(img_array)):
      out.write(img_array[i])
  out.release()



In [144]:
path_walk = '/content/project_data/' 
file_example_path = '/content/project_data/daySequence1/daySequence1/frames/daySequence1--00001.jpg'

list_path_files = list_of_files(path_walk,ftype='BOX.csv')

DF_list = []
for ipath in list_path_files:
  DF_list.append( pd.read_csv(ipath,sep=';'))
result  = pd.concat(DF_list)

fileName = result['Filename'].str.split('/',expand=True)[1]
result['fileName01'] = fileName
dataDo = result[['fileName01','Filename','Annotation tag','Upper left corner X','Upper left corner Y','Lower right corner X','Lower right corner Y']]


img = cv2.imread(file_example_path)
Heig, Wdth, layers = img.shape

dataDo['cntX'] = (dataDo['Upper left corner X']+dataDo['Lower right corner X'])/2/Wdth
dataDo['cntY'] = (dataDo['Upper left corner Y']+dataDo['Lower right corner Y'])/2/Heig
dataDo['Width'] =  abs(dataDo['Upper left corner X']-dataDo['Lower right corner X'])/Wdth
dataDo['Height'] = abs(dataDo['Upper left corner Y']-dataDo['Lower right corner Y'])/Heig
dataDo['lbl'] = 0
dataDo['lbl'][dataDo['Annotation tag']=='stop'] =  1
dataDo['lbl'][dataDo['Annotation tag']=='go'] =  3
dataDo['lbl'][dataDo['Annotation tag']=='warning'] = 2
#dataDo['lbl'][(dataDo['Annotation tag']=='warning')|(dataDo['Annotation tag']=='warning')] = 0  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [145]:
path_to_load = '/content/LISA_res/'

#list_path_files01 = list_of_files(path_walk,ftype='.jpg')
#copyJpgfromList(path_to_load,list_path_files01)
crtTxtFiles(path_to_load,dataDo)
#crtEmptyTxt(path_to_load)

In [136]:
compareTextJpg(path_to_load);

no jpg /content/LISA_res/empty.jpg
24988 24989


In [ ]:
path_to_load = '/content/LISA_res/'
out_video_path= '/content/LISA.avi'
makeVidoeYolo(path_to_load,out_video_path,N_frames=40,fps=1)

In [ ]:
!zip -r /content/drive/MyDrive/TL_project/LISA_res.zip  /content/LISA_res

In [ ]:
! rm -rf /content/LISA_res/

# Работа с ячейкой как с консолью

In [ ]:
!rm -rf /content/content/

In [ ]:
!rm -rf /content/driveU/Dortmund/
!rm -rf /content/driveU/Dortmun_res/

!rm -rf /content/driveU/Frankfurt/
!rm -rf /content/driveU/Frankfurt_res/

!rm -rf /content/driveU/Hannover/
!rm -rf /content/driveU/Hannover_res/

!rm -rf  /content/driveU/Fulda/
!rm -rf  /content/driveU/Fulda_res/